In [1]:
# %%
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def depth_read(filename):
    depth_png = np.array(Image.open(filename), dtype=int)
    assert(np.max(depth_png) > 255)
    depth = depth_png.astype(float) / 256.  # 或者使用 np.float64
    depth[depth_png == 0] = -1.
    return depth

def process_file(file_path):
    try:
        depth = depth_read(file_path)
        return depth[depth != -1].flatten()
    except AssertionError:
        return []

# Set the directory path
directory_path = '/data1/Chenbingyuan/Depth-Completion/g2_dataset/scale/train'
pattern = f"{directory_path}/**/*groundtruth/image_02/**/*.png"
file_paths = glob.glob(pattern, recursive=True)

depth_values = []

with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_file, file_path) for file_path in file_paths]
    for future in tqdm(as_completed(futures), total=len(futures)):
        depth_values.extend(future.result())

depth_values_np = np.array(depth_values) /85. * 100.
depth_values_np = np.clip(depth_values_np, 0, 100)
if depth_values_np.size > 0:
    min_depth = np.min(depth_values_np)
    max_depth = np.max(depth_values_np)
    mean_depth = np.mean(depth_values_np)
    std_depth = np.std(depth_values_np)
else:
    min_depth, max_depth, mean_depth, std_depth = (None, None, None, None)
print('KITTI')
print(f'min_depth={min_depth}, max_depth={max_depth}, mean_depth={mean_depth}, std_depth={std_depth}')


# %%





100%|██████████| 3555/3555 [00:32<00:00, 109.24it/s]


KITTI
min_depth=2.380514705882353, max_depth=100.0, mean_depth=20.14168251868549, std_depth=14.821002445275797


In [7]:
# %%
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

DATASET_LIST =  ['nyu', 'redweb','ETH3D','Ibims', 'VKITTI', 'Matterport3D', 'UnrealCV']

def depth_read(filename):
    # Read Depth and converts to 0-1
    str_file = str(filename)
    if '.png' in str_file:
        data = Image.open(filename)
        
        if 'DIODE' in str_file:
            if '_indoors_' in str_file:
                depth = (np.array(data) / 20.).astype(np.float32)
            elif '_outdoor_' in str_file:
                depth = (np.array(data) / 100.).astype(np.float32)
            depth = np.clip(depth, 0, 1)
        elif 'HRWSI' in str_file:
            depth = (np.array(data) / 255.).astype(np.float32)
            depth = np.clip(depth, 0, 1)
        elif 'redweb' in str_file:
            depth = (np.array(data) / 255.).astype(np.float32)
            depth = np.clip(depth, 0, 1)
        elif 'NYU' in str_file:
            depth = (np.array(data) / 255.).astype(np.float32)
            depth = np.clip(depth, 0, 1)
        else:
            depth = (np.array(data)/ 65535.).astype(np.float32)
            depth = np.clip(depth, 0, 1)
        data.close()
    return depth

def process_file(file_path):
    try:
        depth = depth_read(file_path)
        return depth[depth != 0].flatten()
    except AssertionError:
        return []
for dataset in DATASET_LIST:
    # Set the directory path
    directory_path = '/data1/Chenbingyuan/Depth-Completion/g2_dataset/' + dataset + '/val/' + dataset + '_gt'
    if dataset == 'nyu':
        directory_path = '/data1/Chenbingyuan/Depth-Completion/g2_dataset/nyu/val/NYUv2_gt'
    pattern = f"{directory_path}/*.png"
    file_paths = glob.glob(pattern, recursive=True)

    depth_values = []

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_file, file_path) for file_path in file_paths]
        for future in tqdm(as_completed(futures), total=len(futures)):
            depth_values.extend(future.result())

    depth_values_np = np.array(depth_values) * 100.
    depth_values_np = np.clip(depth_values_np, 0, 100)

    if depth_values_np.size > 0:
        min_depth = np.min(depth_values_np) 
        max_depth = np.max(depth_values_np)
        mean_depth = np.mean(depth_values_np)
        std_depth = np.std(depth_values_np)
    else:
        min_depth, max_depth, mean_depth, std_depth = (None, None, None, None)
    print(f'dataset={dataset}  mean_depth={mean_depth}')


    # %%





100%|██████████| 200/200 [00:02<00:00, 80.34it/s]


dataset=nyu  mean_depth=27.35395622253418


100%|██████████| 510/510 [00:06<00:00, 74.91it/s] 


dataset=redweb  mean_depth=62.728607177734375


100%|██████████| 908/908 [00:02<00:00, 367.62it/s]


...

dataset=VKITTI  mean_depth=17.471233367919922


100%|██████████| 3814/3814 [00:27<00:00, 138.02it/s]


dataset=Matterport3D  mean_depth=6.4739460945129395


100%|██████████| 2000/2000 [00:18<00:00, 110.59it/s]


dataset=UnrealCV  mean_depth=7.2951741218566895


In [16]:
# 将point有值的地方不纳入尺度计算。
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed



DATASET_LIST =  ['nyu', 'redweb','ETH3D','Ibims', 'VKITTI', 'Matterport3D', 'UnrealCV']
method_list = ['sfv2_DIODE_HRWSI', 'LRRU']
point = '0.001'

def process_file(file_path,method,point):
    point_path = file_path.replace('result/rz_sb_mar_'+method, 'point/point')
    # print(point_path)
    try:
        depth_array = np.array(Image.open(file_path)).astype(np.float32)
        point_array = np.array(Image.open(point_path)).astype(np.float32)
        depth = np.clip(depth_array.astype(np.float32), 0.0, 255.0) / 255.
        depth[point_array!=0] = 0
        return depth[depth != 0].flatten()
    except AssertionError:
        return []
for method in method_list:
    
    for dataset in DATASET_LIST:
        png_dir = '/data1/Chenbingyuan/Depth-Completion/g2_dataset/' + dataset + '/result/rz_sb_mar_' + method + '_'+ point + '/' + dataset +'_gt/'
        if dataset =='nyu':
            png_dir = '/data1/Chenbingyuan/Depth-Completion/g2_dataset/' + dataset + '/result/rz_sb_mar_' + method + '_'+ point + '/' + 'NYUv2' +'_gt/'
        png_files = glob.glob(png_dir + '/**.png', recursive=True)
        depth_values = []

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_file, file_path, method, point) for file_path in png_files]
            for future in tqdm(as_completed(futures), total=len(futures)):
                depth_values.extend(future.result())
        
        depth_values_np = np.array(depth_values) * 100.
        depth_values_np = np.clip(depth_values_np, 0, 100)

        if depth_values_np.size > 0:
            min_depth = np.min(depth_values_np) 
            max_depth = np.max(depth_values_np)
            mean_depth = np.mean(depth_values_np)
            std_depth = np.std(depth_values_np)
        else:
            min_depth, max_depth, mean_depth, std_depth = (None, None, None, None)
        

        print(f'dataset={dataset} method={method}  mean_depth={mean_depth}')




100%|██████████| 100/100 [00:00<00:00, 163.48it/s]


dataset=nyu method=sfv2_DIODE_HRWSI  mean_depth=29.67812156677246


100%|██████████| 255/255 [00:01<00:00, 136.33it/s]


dataset=redweb method=sfv2_DIODE_HRWSI  mean_depth=62.157772064208984


100%|██████████| 454/454 [00:03<00:00, 135.69it/s]


...

dataset=VKITTI method=LRRU  mean_depth=2.23785662651062


100%|██████████| 1907/1907 [00:02<00:00, 823.02it/s]


dataset=Matterport3D method=LRRU  mean_depth=1.108298659324646


100%|██████████| 1000/1000 [00:01<00:00, 656.80it/s]


dataset=UnrealCV method=LRRU  mean_depth=1.4643157720565796


In [15]:
# 将point有值的地方不纳入尺度计算。
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed



DATASET_LIST =  ['nyu', 'redweb','ETH3D','Ibims', 'VKITTI', 'Matterport3D', 'UnrealCV']
method_list = ['CFormer_KITTI', 'PEnet', 'g2_DIODE_HRWSI', 'CFormer_DIODE_HRWSI' ]
point = '0.001'

def process_file(file_path,method,point):
    point_path = file_path.replace('result/rz_sb_mar_'+method, 'point/point')
    # print(point_path)
    try:
        depth_array = np.array(Image.open(file_path)).astype(np.float32)
        point_array = np.array(Image.open(point_path)).astype(np.float32)
        depth = np.clip(depth_array.astype(np.float32), 0.0, 255.0) / 255.
        depth[point_array!=0] = 0
        return depth[depth != 0].flatten()
    except AssertionError:
        return []
for method in method_list:
    
    for dataset in DATASET_LIST:
        png_dir = '/data1/Chenbingyuan/Depth-Completion/g2_dataset/' + dataset + '/result/rz_sb_mar_' + method + '_'+ point + '/' + dataset +'_gt/'
        if dataset =='nyu':
            png_dir = '/data1/Chenbingyuan/Depth-Completion/g2_dataset/' + dataset + '/result/rz_sb_mar_' + method + '_'+ point + '/' + 'NYUv2' +'_gt/'
        png_files = glob.glob(png_dir + '/**.png', recursive=True)
        depth_values = []

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_file, file_path, method, point) for file_path in png_files]
            for future in tqdm(as_completed(futures), total=len(futures)):
                depth_values.extend(future.result())
        
        depth_values_np = np.array(depth_values) * 100.
        depth_values_np = np.clip(depth_values_np, 0, 100)

        if depth_values_np.size > 0:
            min_depth = np.min(depth_values_np) 
            max_depth = np.max(depth_values_np)
            mean_depth = np.mean(depth_values_np)
            std_depth = np.std(depth_values_np)
        else:
            min_depth, max_depth, mean_depth, std_depth = (None, None, None, None)
        

        print(f'dataset={dataset} method={method}  mean_depth={mean_depth}')




100%|██████████| 100/100 [00:00<00:00, 129.19it/s]


dataset=nyu method=CFormer_KITTI  mean_depth=28.287559509277344


100%|██████████| 255/255 [00:01<00:00, 136.68it/s]


dataset=redweb method=CFormer_KITTI  mean_depth=59.719417572021484


100%|██████████| 454/454 [00:03<00:00, 137.42it/s]


...

dataset=VKITTI method=CFormer_DIODE_HRWSI  mean_depth=24.825681686401367


100%|██████████| 1907/1907 [00:14<00:00, 130.31it/s]


dataset=Matterport3D method=CFormer_DIODE_HRWSI  mean_depth=15.474305152893066


100%|██████████| 1000/1000 [00:07<00:00, 125.90it/s]


dataset=UnrealCV method=CFormer_DIODE_HRWSI  mean_depth=15.304646492004395
